In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from scipy.cluster.hierarchy import linkage, dendrogram

# Import Optuna
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import r2_score

from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, r2_score


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import logging
# Set the log level for the optuna package to WARNING
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [2]:
df = pd.read_csv("../artifacts/data.csv")

In [3]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)

In [6]:
test_set

,id,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield
3519,3519,25.0,0.50,0.38,0.38,0.75,77.4,46.8,64.7,55.8,27.0,45.8,1.0,0.10,0.508104,0.454680,42.532221,8711.20896
6096,6096,12.5,0.25,0.25,0.50,0.63,69.7,42.1,58.2,50.2,24.3,41.2,34.0,0.56,0.544643,0.442249,35.343967,5914.16491
895,895,12.5,0.25,0.25,0.25,0.38,94.6,57.2,79.0,68.2,33.0,55.9,34.0,0.56,0.411591,0.409907,31.004606,4234.86859
11345,11345,25.0,0.50,0.38,0.50,0.75,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.496826,0.443962,35.555115,5504.75083
7219,7219,25.0,0.50,0.38,0.38,0.63,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.349908,0.358821,29.344457,3276.36206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14832,14832,25.0,0.25,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.467965,0.430330,30.791256,5323.30034
8271,8271,12.5,0.25,0.25,0.50,0.50,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.595066,0.494589,41.545823,7626.61733
14422,14422,25.0,0.50,0.38,0.50,0.75,94.6,57.2,79.0,68.2,33.0,55.9,24.0,0.39,0.559325,0.471650,37.251242,6595.45629
9239,9239,12.5,0.25,0.25,0.63,0.50,69.7,42.1,58.2,50.2,24.3,41.2,16.0,0.26,0.538371,0.446570,36.130192,6594.04460


In [7]:
def get_new_features(df):
        """
        This function is resonpisble for adding new features required for the model training.
        """
        threshold_high = 72
        threshold_low = 50
        
        # 1. Temperature range
        df['TemperatureRange'] = df['MaxOfUpperTRange'] - df['MinOfLowerTRange']    
        df['ExtremeHighTemp'] = (df['AverageOfUpperTRange'] > threshold_high).astype(int)
        df['ExtremeLowTemp'] = (df['AverageOfLowerTRange'] < threshold_low).astype(int)

        # 2. Total bee density
        df['TotalBeeDensity'] = df['honeybee'] + df['bumbles'] + df['andrena'] + df['osmia']

        # 3. Bee species dominance
        total_density = df['honeybee'] + df['bumbles'] + df['andrena'] + df['osmia']
        df['HoneybeeDominance'] = df['honeybee'] / total_density
        df['BumblesBeeDominance'] = df['bumbles'] / total_density
        df['AndrenaBeeDominance'] = df['andrena'] / total_density
        df['OsmiaBeeDominance'] = df['osmia'] / total_density

        # 4. Rain intensity
        df['RainIntensity'] = df['AverageRainingDays'] / df['RainingDays']

        # 5. Interaction features
        df['BeeDensity_TemperatureInteraction'] = df['TotalBeeDensity'] * df['TemperatureRange']
        df['BeeDensity_RainInteraction'] = df['TotalBeeDensity'] * df['RainIntensity']
        return df        

In [9]:
# Get new features as part of feature engineering
train_df_fe = get_new_features(train_set)
test_df_fe = get_new_features(test_set)

In [12]:
target_column_name = "yield"
id_column_name = "id"
input_feature_train_df = train_df_fe.drop(columns=[target_column_name, id_column_name],axis=1)
target_feature_train_df = train_df_fe[target_column_name]

input_feature_test_df = test_df_fe.drop(columns=[target_column_name, id_column_name],axis=1)
target_feature_test_df = test_df_fe[target_column_name]

In [16]:
input_feature_test_df

,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,...,ExtremeHighTemp,ExtremeLowTemp,TotalBeeDensity,HoneybeeDominance,BumblesBeeDominance,AndrenaBeeDominance,OsmiaBeeDominance,RainIntensity,BeeDensity_TemperatureInteraction,BeeDensity_RainInteraction
3519,25.0,0.50,0.38,0.38,0.75,77.4,46.8,64.7,55.8,27.0,...,0,1,2.01,0.248756,0.189055,0.189055,0.373134,0.100000,101.304,0.201000
6096,12.5,0.25,0.25,0.50,0.63,69.7,42.1,58.2,50.2,24.3,...,0,1,1.63,0.153374,0.153374,0.306748,0.386503,0.016471,74.002,0.026847
895,12.5,0.25,0.25,0.25,0.38,94.6,57.2,79.0,68.2,33.0,...,1,0,1.13,0.221239,0.221239,0.221239,0.336283,0.016471,69.608,0.018612
11345,25.0,0.50,0.38,0.50,0.75,86.0,52.0,71.9,62.0,30.0,...,0,0,2.13,0.234742,0.178404,0.234742,0.352113,0.016250,119.280,0.034612
7219,25.0,0.50,0.38,0.38,0.63,86.0,52.0,71.9,62.0,30.0,...,0,0,1.89,0.264550,0.201058,0.201058,0.333333,0.016250,105.840,0.030713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14832,25.0,0.25,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,...,0,0,1.00,0.250000,0.250000,0.250000,0.250000,0.016250,56.000,0.016250
8271,12.5,0.25,0.25,0.50,0.50,86.0,52.0,71.9,62.0,30.0,...,0,0,1.50,0.166667,0.166667,0.333333,0.333333,0.016250,84.000,0.024375
14422,25.0,0.50,0.38,0.50,0.75,94.6,57.2,79.0,68.2,33.0,...,1,0,2.13,0.234742,0.178404,0.234742,0.352113,0.016250,131.208,0.034612
9239,12.5,0.25,0.25,0.63,0.50,69.7,42.1,58.2,50.2,24.3,...,0,1,1.63,0.153374,0.153374,0.386503,0.306748,0.016250,74.002,0.026488


In [18]:
scaler = StandardScaler()

In [19]:
input_feature_train_arr = scaler.fit_transform(input_feature_train_df)
input_feature_test_arr = scaler.transform(input_feature_test_df)

In [21]:
train_arr = np.c_[input_feature_train_arr, np.array(target_feature_train_df)]
test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]

In [23]:
train_arr[:,:-1]

array([[ 0.80843626,  0.30828629,  1.54924349, ..., -0.54826283,
        -0.11897742, -0.47045015],
       [-1.09632254, -0.38661677, -0.61812629, ..., -0.54200074,
        -0.97249496, -0.69198322],
       [-1.09632254, -0.38661677,  1.54924349, ..., -0.54826283,
        -0.47787591, -0.56780924],
       ...,
       [ 0.80843626,  0.30828629, -0.61812629, ..., -0.54826283,
         0.43407797, -0.50375721],
       [-1.09632254, -0.38661677, -0.61812629, ...,  1.82924431,
        -0.07378981,  1.37956949],
       [ 0.80843626,  0.30828629,  1.54924349, ..., -0.54826283,
         0.45615918, -0.50119513]])

In [24]:
train_arr[:,-1]

array([4177.0152 , 3238.02815, 7451.72563, ..., 5892.161  , 7231.02778,
       6696.23571])

In [25]:
X_train, y_train, X_test, y_test = (
                train_arr[:,:-1],
                train_arr[:,-1],
                test_arr[:,:-1],
                test_arr[:,-1],
            )

In [27]:
def lgbm_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 500)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    num_leaves = trial.suggest_int("num_leaves", 31, 127)
    subsample = trial.suggest_float("subsample", 0.5, 1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1)

    model = LGBMRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        num_leaves=num_leaves,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=42
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2

def catboost_objective(trial):
    iterations = trial.suggest_int("iterations", 100, 500)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2)
    depth = trial.suggest_int("depth", 3, 10)
    l2_leaf_reg = trial.suggest_int("l2_leaf_reg", 1, 9)
    subsample = trial.suggest_float("subsample", 0.5, 1)

    model = CatBoostRegressor(
        iterations=iterations,
        learning_rate=learning_rate,
        depth=depth,
        l2_leaf_reg=l2_leaf_reg,
        subsample=subsample,
        random_state=42,
        verbose=0
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2

In [30]:
# Run Optuna optimization for each model
sampler = TPESampler(seed=42)

lgbm_study = optuna.create_study(direction="maximize", sampler=sampler)
lgbm_study.optimize(lgbm_objective, n_trials=50)

catboost_study = optuna.create_study(direction="maximize", sampler=sampler)
catboost_study.optimize(catboost_objective, n_trials=50)

# Print best hyperparameters for each model from Optuna
# print("Best parameters for XGBoost from Optuna: ", xgb_study.best_params)
print("Best parameters for LightGBM from Optuna: ", lgbm_study.best_params)
print("Best parameters for CatBoost from Optuna: ", catboost_study.best_params)

Best parameters for LightGBM from Optuna:  {'n_estimators': 136, 'learning_rate': 0.03856943497773353, 'max_depth': 5, 'num_leaves': 109, 'subsample': 0.7653310551382453, 'colsample_bytree': 0.9694417669567851}
Best parameters for CatBoost from Optuna:  {'iterations': 425, 'learning_rate': 0.03518043260586421, 'depth': 7, 'l2_leaf_reg': 2, 'subsample': 0.9053149480681327}


In [31]:
# Train multiple base models with the best parameters
# xgb = XGBRegressor(**xgb_best_params, random_state=42)
lgbm = LGBMRegressor(**lgbm_study.best_params, random_state=42)
catboost = CatBoostRegressor(**catboost_study.best_params, random_state=42, verbose=0)

In [34]:
# Feature selection using SelectKBest
selector = SelectKBest(score_func=f_regression, k=10)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

In [35]:
# Combine the base models to create an ensemble and assign weights
weights = [0.3, 0.3]
# Combine the base models to create an ensemble
ensemble = VotingRegressor([('lgbm', lgbm), ('catboost', catboost)])
# Train the ensemble model using selected features
ensemble.fit(X_train_selected, y_train)

VotingRegressor(estimators=[('lgbm',
                             LGBMRegressor(colsample_bytree=0.9694417669567851,
                                           learning_rate=0.03856943497773353,
                                           max_depth=5, n_estimators=136,
                                           num_leaves=109, random_state=42,
                                           subsample=0.7653310551382453)),
                            ('catboost',
                             <catboost.core.CatBoostRegressor object at 0x00000241728C1F10>)])

In [36]:
# Cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(ensemble, X_train_selected, y_train, cv=kfold, scoring='r2')
print("Cross-validation scores: ", cv_scores)
print("Mean CV R-squared: {:.2f}".format(np.mean(cv_scores)))
print("Standard Deviation of CV R-squared: {:.2f}".format(np.std(cv_scores)))

Cross-validation scores:  [0.78994678 0.82385479 0.82125084 0.8357447  0.81590599]
Mean CV R-squared: 0.82
Standard Deviation of CV R-squared: 0.02


In [37]:
# Make predictions using the selected features
y_pred = ensemble.predict(X_test_selected)

# Evaluate the performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error: {:.2f}".format(mse))
print("R-squared: {:.2f}".format(r2))

Mean Squared Error: 312265.35
R-squared: 0.82


In [53]:
cols_idxs = selector.get_support(indices=True)
cols_idxs

array([ 0, 11, 12, 13, 14, 15, 20, 23, 24, 26], dtype=int64)

In [54]:
features_df_new = train_set.iloc[:,cols_idxs]
features_df_new.head()

,id,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,ExtremeLowTemp,BumblesBeeDominance,AndrenaBeeDominance,RainIntensity
245,245,41.2,24.0,0.39,0.384981,0.399724,1,0.189055,0.248756,0.016250
3017,3017,55.9,34.0,0.56,0.335530,0.376874,0,0.221239,0.336283,0.016471
8047,8047,45.8,24.0,0.39,0.582483,0.488569,1,0.233129,0.306748,0.016250
14223,14223,55.9,1.0,0.10,0.573428,0.466671,0,0.202128,0.265957,0.100000
13397,13397,55.9,16.0,0.26,0.502398,0.453650,0,0.117371,0.295775,0.016250


In [56]:
X_train.shape

(12231, 27)

In [57]:
features_df_new.shape

(12231, 10)

In [59]:
train_set.shape

(12231, 29)